# SE/2015/026

# S. PRASANTH

## PREDICT AND GIVE QUANTITATIVE SCORE FOR COVID-19 USING CHEST CT IMAGES

In [1]:
# pip install pydicom

In [2]:
import pydicom
import os
import pandas as pd

In [3]:
metaData = pd.read_csv('../Dataset/CSV/CT Metadata.csv', index_col=1)

metaData.head()

,Id,Label,Dataset_type,Result
X_ray_image_name,,,,
ARDS (1).jpg,0,ARDS,TRAIN,0
ARDS (2).jpg,1,ARDS,TRAIN,0
Covid-19 (1).jpeg,2,COVID-19,TRAIN,1
Covid-19 (2).jpeg,3,COVID-19,TRAIN,1
Covid-19 (3).jpeg,4,COVID-19,TRAIN,1


## DATA PRE-PROCESSING

In [4]:
#Check for Null Values
metaData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5953 entries, ARDS (1).jpg to SARS (11).jpeg
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            5953 non-null   int64 
 1   Label         5953 non-null   object
 2   Dataset_type  5953 non-null   object
 3   Result        5953 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 232.5+ KB


In [5]:
#Check for Data duplications
metaData.duplicated().sum()

0

In [14]:
metaSummaryData = pd.read_csv('../Dataset/CSV/CT Metadata Summary.csv', index_col=1)

metaSummaryData.head(8)

,Label
Image_Count,
117,COVID-19
1576,Normal
2,ARDS
2773,Pnemonia Bacteria
1469,Pnemonia Virus
11,SARS
5,Streptococcus
5953,Total


In [6]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

In [7]:
IMG_PX_SIZE = 50
HM_SLICES = 20

In [8]:
def Chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [9]:
def Mean(l):
    return sum(l)/len(l)

In [10]:
def ProcessData(patient, labels_df, img_px_size = 50, hm_slices = 20, visualize = False):
    label = label_df.get_value(patient, 'covid')
    path = data_dir + patient